# Advent of Code, day 1
## Part 1

Data is in a text file named `day_01.txt` in the data dir. A cursory look at the file shows blocks of numbers are separated by double line break, so we can read and split on that separator before doing anything else.

In [1]:
with open('data/day_01.txt', 'r') as infile:
    blocks = infile.read().split('\n\n') 
blocks[:3]

['2832\n2108\n3082\n4328\n6843\n5121\n2869\n1366\n2358\n1680\n4980\n1161',
 '8026\n2154\n4242\n1023\n2744\n3162\n4093\n1150\n5397\n2738\n5657',
 '10954\n11208\n8034\n1636\n9430\n9421\n5025']

Alright, now we have each block as a list item, but they're still strings with line breaks as delimiters between the integers.

Let's try a few different methods to split, cast to int, sum, and take the maximum of these blocks of numbers. We'll use the `%timeit` magic to see which method is fastest.

In [2]:
# all maps, using an anonymous function for mapping the split blocks to integers
%timeit max(map(sum, tuple(map(lambda block: map(int, block.split()), blocks))))
max(map(sum, tuple(map(lambda block: map(int, block.split()), blocks))))

217 µs ± 9.22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


67622

In [3]:
# outer map, but using a tuple comprehension to cast the split blocks to integers
%timeit max(map(sum, (map(int, block.split()) for block in blocks)))

209 µs ± 9.79 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
# nested tuple comprehensions
%timeit max((sum((int(num) for num in block.split())) for block in blocks))

259 µs ± 2.38 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Looks like a mix of map and tuple comprehension is actually fastest, potentially because the lambda in the all-map solution introduces some additional overhead that the map/tuple comprehension combination does not.

The nested tuple comprehension solution is slowest, perhaps because allocation isn't optimal when using lists of varying lengths?

## Part 2

In [5]:
# sorting
%timeit sum(sorted(tuple(map(sum, (map(int, block.split()) for block in blocks))))[-3:])
sum(sorted(tuple(map(sum, (map(int, block.split()) for block in blocks))))[-3:])

222 µs ± 16.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


201491

In [6]:
%%timeit import numpy as np
# numpy argmax
arr = np.array(tuple(map(sum, (map(int, block.split()) for block in blocks))))
arr[np.argpartition(arr, -3)[-3:]].sum()

395 µs ± 118 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%%timeit import numpy as np
# numpy sort
np.sort(tuple(map(sum, (map(int, block.split()) for block in blocks))))[-3:].sum()

238 µs ± 19.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


The numpy argmax has better time complexity, in theory, than the numpy sort, but the additional variable assignment that we need to do probably introduces a bit of extra overhead, making it a wash for this comparatively small array.

The base Python sort is still the fastest option.

For both Part 1 and Part 2 it's possible that a highly optimized solution for this specific problem (e.g. iterating over the blocks and storing the three highest values) is faster for (very) large datasets, but I'm not really interested in that kind of optimization, mostly because it tends to produce fragile algorithms.